**NOTE** `phimats` environment should be used as kernel

In [19]:
import numpy as np

from PreProcessing import PhysicsConfig, MeshConfig, PreProcessing as PP

from MeshManager import MeshManager

from BoundaryConditions import *

from PostProcessing import WriteXDMF

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Simulation data

In [20]:
SimulName = "SENT_EPTH"
# Element sets
nElementSets = 1
# Number of steps to achieve the load
nSteps = 200

### Read mesh file

In [21]:
# Element name
elementName = "quad"  		# meshio compatible element name
mesh = MeshManager("SENT.msh", elementName)
mesh.WriteMesh(SimulName)

Reading mesh: SENT.msh...

Mesh consistency verified: All domain elements are <quad>.

PHIMATS MESH INITIATION
Detected 1 Material Sets. Follow this order in your Input Data:
  [Index 1] -> Physical Group: 'SENT'

------------------------------
MESH SUMMARY
------------------------------
File Name:      SENT.msh
Element Type:   quad
Dimension:      2D
Node Order:     1
Total Nodes:    26332
Total Elements: 25988
Cell Blocks:
  - vertex: 1 items
  - line: 20 items
  - quad: 25988 items
------------------------------



In [22]:
# Create the config object
meshConfig = MeshConfig(
    nTotNodes=mesh.get_nTotNodes(),
    nTotElements=mesh.get_nTotElements(),
    nDim=mesh.get_nDim(),
    materialNames=mesh.getMaterialNames(),
)

### Apply mechanics boundary conditions
**NOTE** This is the total load to be achieved in `nSteps`.

In [23]:
# Dirichlet BCs list
presBCs = []

# Top pull
presBCs.extend(AssignDirichletBC(mesh.getNodesByGroup("top"), dof=1, value=2.2e-4))

# Bottom fix
presBCs.extend(AssignDirichletBC(mesh.getNodesByGroup("bottom"), dof=1, value=0.0))

# Bottom left corner pin (Fixed in X)
presBCs.extend(AssignDirichletBC(mesh.getNodesByGroup("corner"), dof=0, value=0.0))

# Write external 
WriteBCVTK(SimulName, mesh, presBCs, dofNames=["ux", "uy"])

BC visualization written to SENT_EPTH_BC.vtu


### Mechanical material data

In [24]:
# Analysis type
AnalysisType = "PlaneStrainPFF"
# Isotropy
Isotropy = "Isotropic"
# Young's modulus
Emod = 210e9
# Poisson's ratio
nu = 0.3

# Plasticity type
Plasticity = "IsoHard"
HardeningLaw = "PowerLaw"
sig_y0 = 500e6    # Pa
K_hard = 885.5e6    # Pa
n_pow = 0.097

# Dislocation evolution (only if you need the normalized dislocation density output, otherwise, set them to zero)

rho_0 = 1e10  # Initial dislocation density (m⁻²)
M = 3 		  # Taylor factor
alpha = 0.3   # Dislocation interaction constant
b = 2.5e-10   # Burgers vector (m)
G = Emod/(2*(1+nu)) # Shear modulus
k1 = 2*(G/(200*G*alpha*b))  # Multiplication coefficient
k2 = 10  # Recovery coefficient

MechMaterial = {
	"SENT" : {
		"Elastic" : {
			"AnalysisType" : AnalysisType,
			"Isotropy" 	 : Isotropy,
			"Emod" 		 : Emod,
			"nu"   		 : nu,
		},
		"Plastic" : {
			"Plasticity"   : Plasticity,
			"HardeningLaw" : HardeningLaw,
			"sig_y0"       : sig_y0,
			"K_hard"       : K_hard,
			"n_pow"        : n_pow,
   			"rho_0"        : rho_0,
			"M"            : M,
			"b"            : b,
			"alpha"        : alpha,
			"k1"           : k1,
   			"k2"           : k2,
		},
	},
}

### PFF material data

In [25]:
const_ell = 3e-5     # Length-scale regularization parameter [m]
wc = 11e6            # Critical work density [J/m³]
Gc = 2*const_ell*wc	 # Fracture toughness [J/m²]

PFFMaterial = {
	"SENT" : {
		"PFF" : {
			"const_ell" : const_ell,
			"wc" 	    : wc,
		}
    }
}

print("l: ", const_ell, " m")
print("Gc: ", Gc, " J/m²")
print("wc: ", wc, " J/m³")

l:  3e-05  m
Gc:  660.0  J/m²
wc:  11000000.0  J/m³


### Initialize simulation object
#### Mechanical

In [26]:
# Create the config object
mechPhysConfig = PhysicsConfig(
    SimulName = SimulName,
    PhysicsType = "Mechanical",
    PhysicsCategory = "Plastic",
    nSteps    = nSteps,
    presBCs=presBCs
)

In [27]:
MechNotch = PP(mechPhysConfig, meshConfig, MechMaterial)


PHIMATS: SENT_EPTH | Mechanical (Plastic)
  Nodes/Elems : 26332 / 25988
  Total DOFs  : 52664
  BCs Count   : 23



In [28]:
MechNotch.WriteInputFile()

  Input file initialized: SENT_EPTH.mech.in.hdf5


#### PFF

In [29]:
# Create the config object
pffPhysConfig = PhysicsConfig(
    SimulName = SimulName,
    PhysicsType = "PFF",
    PhysicsCategory = list(PFFMaterial["SENT"].keys())[0],
    nSteps    = nSteps,
)

In [30]:
PFFNotch = PP(pffPhysConfig, meshConfig, PFFMaterial)


PHIMATS: SENT_EPTH | PFF (PFF)
  Nodes/Elems : 26332 / 25988
  Total DOFs  : 26332
  BCs Count   : 0



In [31]:
PFFNotch.WriteInputFile()

  Input file initialized: SENT_EPTH.pff.in.hdf5


### Write output files

In [34]:
MechNotch.WriteOutputFile(OVERWRITE=True)
PFFNotch.WriteOutputFile(OVERWRITE=True)

  Output file initialized: SENT_EPTH.mech.out.hdf5
  Output file initialized: SENT_EPTH.pff.out.hdf5


### Write .xdmf visualization

In [33]:
WriteXDMF(SimulName, # Base simulation name
          elementName, # Element name
          nSteps+1, # Number of steps 
          components=["mech", "pff"],
    nDim=2,
    mechModel="Plastic", # Adds strain_p, stress_eq, etc.
    )